In [59]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from datetime import datetime, date, timedelta
import math
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

import seaborn as sns
sns.set()
sns.set_theme()

import statistics as s 

In [60]:
# Lectura de datos
start = datetime(1900,1,1)
end = datetime.now()
data = pdr.get_data_yahoo('BTC-USD', start, end, interval='d')
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,2.105680e+07,457.334015
2014-09-18,456.859985,413.104004,456.859985,424.440002,3.448320e+07,424.440002
2014-09-19,427.834991,384.532013,424.102997,394.795990,3.791970e+07,394.795990
2014-09-20,423.295990,389.882996,394.673004,408.903992,3.686360e+07,408.903992
2014-09-21,412.425995,393.181000,408.084991,398.821014,2.658010e+07,398.821014
...,...,...,...,...,...,...
2021-11-17,60823.609375,58515.410156,60139.621094,60368.011719,3.917839e+10,60368.011719
2021-11-18,60948.500000,56550.792969,60360.136719,56942.136719,4.138834e+10,56942.136719
2021-11-19,58351.113281,55705.179688,56896.128906,58119.578125,3.870241e+10,58119.578125


In [61]:
def cagr(df):
    """
    cagr [retorna el CAGR en un DataFrame]
    [Una tasa de crecimiento anual compuesto (CAGR) mide la tasa de retorno de una inversión (como un fondo de inversión o un bono) durante un período de inversión, como 5 o 10 años]
    Args:
        df ([DataFrame]): [DataFrame obtenido al leer los datos del activo de interés]
    """
    df = df.copy()
    df = df.sort_index(ascending=True)
    df['cum_return'] = (1 + df['btc_return']).cumprod()
    total_ret = df['cum_return'][-1]

    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years)-1
    return CAGR

In [62]:
# Crear variables y subset para análisis
df = data[['Adj Close']]['2016':'2021'].copy()
df = df.rename(columns={'Adj Close': 'btc_price'})
df['btc_return'] = df['btc_price'].pct_change()
df['day'] = df.index.day
df['weekday'] = df.index.dayofweek
df['month'] = df.index.month
df['year'] = df.index.year
df.head()

,btc_price,btc_return,day,weekday,month,year
Date,,,,,,
2016-01-01,434.334015,NaN,1,4,1,2016
2016-01-02,433.437988,-0.002063,2,5,1,2016
2016-01-03,430.010986,-0.007907,3,6,1,2016
2016-01-04,433.091003,0.007163,4,0,1,2016
2016-01-05,431.959991,-0.002611,5,1,1,2016


In [63]:

def DCA_variable(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la variación del retorno y la cantidad de btc obtenidos
   '''
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < -0.0):
         df.at[x,'usdt_invested'] = usdt*(1+row['btc_return'])
      else:
         df.at[x, 'usdt_invested'] = usdt
   df['btc_amount'] = df_sem['usdt_invested']/df_sem['btc_price']
   return df.head(20)

In [64]:
#Estrategia aplicando DCA variable con compra mayor en caídas
df_sem = df[df['weekday']==1]
df_sem['btc_return'] = df_sem['btc_price'].pct_change()
DCA_variable(df_sem, 50)

,btc_price,btc_return,day,weekday,month,year,usdt_invested,btc_amount
Date,,,,,,,,
2016-01-05,431.959991,NaN,5,1,1,2016,50.000000,0.115751
2016-01-12,435.690002,0.008635,12,1,1,2016,50.000000,0.114760
2016-01-19,380.148987,-0.127478,19,1,1,2016,43.626086,0.114760
2016-01-26,392.153015,0.031577,26,1,1,2016,50.000000,0.127501
2016-02-02,374.447998,-0.045148,2,1,2,2016,47.742588,0.127501
2016-02-09,376.028992,0.004222,9,1,2,2016,50.000000,0.132968
2016-02-16,407.488007,0.083661,16,1,2,2016,50.000000,0.122703
2016-02-23,420.735992,0.032511,23,1,2,2016,50.000000,0.118839
2016-03-01,435.122986,0.034195,1,1,3,2016,50.000000,0.114910


In [65]:
def calculate_return(df):
    total_btc_amount = df_sem['btc_amount'].sum()
    total_usdt_invested = df_sem['usdt_invested'].sum()
    total_usdt_obtained = round(total_btc_amount*df['btc_price'][-1],3)
    total_return = (total_usdt_obtained/total_usdt_invested-1)*100
    cagr_value = cagr(df_sem)
    print(f'total_return: {round(total_return,2)}%')
    print(f'total_usdt_invested: {total_usdt_invested}')
    print(f'total_usdt_obtained: {total_usdt_obtained}')
    print(f'cagr: {round(cagr_value,2)}')

In [66]:
calculate_return(df_sem)

total_return: 2782.64%
total_usdt_invested: 14869.728804359707
total_usdt_obtained: 428640.877
cagr: 1.32
